In [1]:
import logging
from prepare_data import HCDALoader
from sklearn.model_selection import KFold

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
loader = HCDALoader()

# load index values from main table
app_ix = loader.applications_train_index()

2018-06-24 11:38:44,844 - DEBUG - Reading application_train.csv...
2018-06-24 11:38:51,482 - DEBUG - Finished reading application_train.csv


In [3]:
# fit model using k-fold verification
kf = KFold(n_splits=2, shuffle=True)
for fold_indexes in kf.split(app_ix):
    print(fold_indexes[0].shape, fold_indexes[1].shape)
    data_train, target_train, data_val, target_val = loader.load_train_val(fold_indexes[0], fold_indexes[1])

(153755,) (153756,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


(153756,) (153755,)


In [4]:
data_train.shape, target_train.shape, data_val.shape, target_val.shape

((153756, 238), (153756,), (153755, 238), (153755,))

In [4]:
#from imblearn.combine import SMOTEENN

#smote_enn = SMOTEENN()
#data_train_os, target_train_os = smote_enn.fit_sample(data_train, target_train)

In [5]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
data_train_os, target_train_os = ros.fit_sample(data_train, target_train)

In [6]:
import numpy as np

np.unique(target_train_os, return_counts=True)

(array([0, 1]), array([212003, 212003]))

In [7]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=data_train_os.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Using TensorFlow backend.


In [8]:
model.fit(data_train_os, target_train_os, epochs=5, batch_size=32, validation_data=(data_val, target_val))

Train on 424006 samples, validate on 76877 samples
Epoch 1/5
424006/424006 [==============================] - 23s 54us/step - loss: 0.6005 - acc: 0.6790 - val_loss: 0.5867 - val_acc: 0.6946
Epoch 2/5
424006/424006 [==============================] - 24s 56us/step - loss: 0.5797 - acc: 0.6979 - val_loss: 0.5731 - val_acc: 0.7044
Epoch 3/5
424006/424006 [==============================] - 24s 55us/step - loss: 0.5693 - acc: 0.7057 - val_loss: 0.5536 - val_acc: 0.7199
Epoch 4/5
424006/424006 [==============================] - 24s 56us/step - loss: 0.5599 - acc: 0.7129 - val_loss: 0.5982 - val_acc: 0.6834
Epoch 5/5
424006/424006 [==============================] - 24s 56us/step - loss: 0.5512 - acc: 0.7198 - val_loss: 0.5598 - val_acc: 0.7189


In [9]:
data_train.dtypes

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'